In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import pandas as pd
from datasets import load_dataset, load_metric
from sklearn.utils import shuffle
import torch 
import mlflow
import numpy as np
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments

C:\Users\nkhozin\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metric = load_metric('rouge')

C:\Users\nkhozin\AppData\Local\Temp\ipykernel_15328\1264828061.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [3]:
model_name = "facebook/bart-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [4]:
train_small = pd.read_csv('data\\train_small.csv', index_col=0)
val = pd.read_csv('data\\eval.csv', index_col=0)

train_pairs = list(zip(train_small.source, train_small.target))
eval_pairs = list(zip(val.source, val.target))
print(train_small.shape[0], val.shape[0])

20480 2560


In [5]:
class SimplificationDataset(torch.utils.data.Dataset):
    def __init__(self, pairs, tokenizer, max_length):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, index):
        source_text, target_text = self.pairs[index]
        source_encoding = self.tokenizer(source_text, truncation=True, max_length=self.max_length, padding="max_length")
        target_encoding = self.tokenizer(target_text, truncation=True, max_length=self.max_length, padding="max_length")

        return {
            "input_ids": source_encoding["input_ids"],
            "attention_mask": source_encoding["attention_mask"],
            "labels": target_encoding["input_ids"],
        }

In [6]:
max_length = 512
train_dataset = SimplificationDataset(train_pairs, tokenizer, max_length=max_length)
eval_dataset = SimplificationDataset(eval_pairs, tokenizer, max_length=max_length)

In [7]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [8]:
def compute_rouge(pred):
    predictions, labels = pred

    decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)

    res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

    pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    res['gen_len'] = np.mean(pred_lens)

    return {k: round(v, 4) for k, v in res.items()}

In [12]:
args = Seq2SeqTrainingArguments(
    'bart', 
    learning_rate=2e-5,
    per_device_train_batch_size= 6,
    per_device_eval_batch_size= 6,
    num_train_epochs=3,
    predict_with_generate=True,
    evaluation_strategy='steps',
    logging_steps=1000,
    save_steps=10000,
    fp16=True
    )

trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_rouge
)

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [13]:
experiment_name = "bart"
mlflow.set_experiment(experiment_name=experiment_name)
current_experiment = dict(mlflow.get_experiment_by_name(experiment_name))
exp_id = current_experiment['experiment_id']

In [14]:
with mlflow.start_run(experiment_id=exp_id, run_name='first_test'):
    # Логируем параметры
    mlflow.log_param('model_name', model_name)
    #mlflow.log_param('max_length', max_length)
    mlflow.log_param('train_data_size', len(train_dataset))
    mlflow.log_param('batch_size', args.per_device_train_batch_size)
    
    trainer.train()
    
    # Логируем метрики
    metrics = trainer.evaluate()
    for key, value in metrics.items():
        mlflow.log_metric(key, value)

    # Сохраняем модель и логируем путь к ней
    output_dir = './bart'
    trainer.save_model(output_dir)
    mlflow.log_artifacts(output_dir, artifact_path='models')
    
mlflow.end_run()

***** Running training *****
  Num examples = 20480
  Num Epochs = 3
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 10242
  Number of trainable parameters = 406291456


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1000,0.190500,0.193545,1.708700,0.364600,1.721600,1.734200,19.994100
2000,0.176300,0.189292,1.612400,0.343700,1.623500,1.641000,19.994100
3000,0.188500,0.174543,1.678200,0.289100,1.700000,1.707300,19.987900
4000,0.186400,0.171757,1.631600,0.317700,1.626000,1.643300,19.992600
5000,0.178800,0.167954,1.685200,0.352300,1.675600,1.696200,19.988300
6000,0.177100,0.164487,1.860900,0.338500,1.867500,1.890300,19.992200
7000,0.172800,0.162993,1.612100,0.303400,1.609900,1.625300,19.989100
8000,0.168700,0.161706,1.620700,0.260400,1.626600,1.631800,19.991400
9000,0.166700,0.159812,1.858800,0.331400,1.867900,1.886700,19.991000
10000,0.164700,0.158522,1.752000,0.270800,1.763600,1.778300,19.994500


***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
***** Running Evaluation *****
  Num examples = 2560
  Batch size = 6
Saving model checkpoint to bart\checkpoint-10000
Configuration saved in bart\checkpoint-10000\config.json
Model weights saved in bart\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in bart\checkpoint-10000\tokenizer_config.json
Special tokens file saved in bart\checkpoint-10000\speci

Saving model checkpoint to ./bart
Configuration saved in ./bart\config.json
Model weights saved in ./bart\pytorch_model.bin
tokenizer config file saved in ./bart\tokenizer_config.json
Special tokens file saved in ./bart\special_tokens_map.json
